In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [5]:
pd.set_option('display.max_colwidth', None)

In [9]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd

# Setup browser
driver = webdriver.Chrome()
url = "https://www.allobank.com/help"
driver.get(url)
time.sleep(5)  # Tunggu halaman awal termuat

# Temukan semua tombol kategori/tab
tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")

categories = []
questions = []
answers = []

# Klik satu per satu tab untuk mengaktifkan dan ambil kontennya
for i in range(len(tab_buttons)):
    try:
        # Cari ulang tombol tiap iterasi (harus fresh karena DOM berubah setelah klik)
        tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")
        category_text = tab_buttons[i].text.strip()
        
        # Klik tab
        tab_buttons[i].click()
        time.sleep(2)  # Tunggu isi tab termuat

        # Ambil HTML baru setelah tab diklik
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        
        # Temukan semua kartu FAQ di tab yang sedang aktif
        faq_cards = soup.find_all("div", class_="cards")
        
        for card in faq_cards:
            question_el = card.find("div", class_="cards__title")
            answer_el = card.find("div", class_="cards__desc")
            
            questions.append(question_el.get_text().strip() if question_el else None)
            answers.append(answer_el.get_text().strip() if answer_el else None)
            categories.append(category_text)  # Gunakan nama tab yang sedang aktif

    except Exception as e:
        print(f"Tab ke-{i} gagal diakses: {e}")
        continue

# Buat DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({
    'category': categories,
    'question': questions,
    'answer': answers
})

df


,category,question,answer
0,UMUM,Apa itu Allo Bank?,"Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan ""All in One"" dan mempunyai arti sebagai ""one for all, and all for one"" alias semua untuk satu dan satu untuk semua."
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,None
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?,None
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan aman?,None
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,None
...,...,...,...
364,REGISTRASI,Mengapa username dan Password saya diinfokan salah disaat saya telah menginput informasi yang benar?,None
365,REGISTRASI,Apa yang harus saya lakukan apabila saya tidak bisa menekan tombol 'Login'?,None
366,REGISTRASI,Apa yang harus saya lakukan apabila saat Login yang muncul hanya layar putih?,None
367,REGISTRASI,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.,None


In [21]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup Chrome WebDriver
driver = webdriver.Chrome()
url = "https://www.allobank.com/help"
driver.get(url)
wait = WebDriverWait(driver, 10)
time.sleep(5)  # Tunggu awal page load

categories = []
questions = []
answers = []

# Ambil semua tab kategori FAQ
tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")

for i in range(len(tab_buttons)):
    try:
        # Reload ulang tab (karena DOM bisa berubah setelah klik)
        tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")
        tab = tab_buttons[i]
        category_text = tab.text.strip()

        # Scroll agar tab terlihat dan klik dengan JavaScript (hindari intercept error)
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(3)
        driver.execute_script("arguments[0].click();", tab)
        time.sleep(2)  # Tunggu konten tab termuat

        # Klik semua pertanyaan agar jawaban muncul
        question_elements = driver.find_elements(By.CLASS_NAME, "cards__title")
        print(f"Total pertanyaan ditemukan: {len(question_elements)}")  # Debug: Cek jumlah pertanyaan
        for q_el in question_elements:
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", q_el)
                wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "cards__title")))
                driver.execute_script("arguments[0].click();", q_el)
                time.sleep(2)  # Tunggu animasi jawaban terbuka
            except Exception as e:
                print(f"❌ Gagal klik pertanyaan: {e}")
                continue

        # Tunggu semua jawaban muncul sebelum ambil HTML
        time.sleep(8)  # Tambahkan waktu untuk menunggu lebih lama
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "cards__desc")))

        # Ambil HTML setelah semua pertanyaan dibuka
        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        faq_cards = soup.find_all("div", class_="cards")

        print(f"Jumlah cards yang ditemukan: {len(faq_cards)}")  # Debug: Cek jumlah cards yang ditemukan

        for card in faq_cards:
            question_el = card.find("div", class_="cards__title")
            answer_el = card.find("div", class_="cards__desc")

            # Cek apakah answer_el ditemukan, jika tidak coba menggunakan elemen lain
            if not answer_el:
                # Jika tidak ditemukan, coba ambil jawaban dari elemen yang lain
                answer_el = card.find("div", class_="some-other-class")  # Misal, coba kelas lain jika ada

            question = question_el.get_text(strip=True) if question_el else None
            answer = answer_el.get_text(strip=True) if answer_el else None

            categories.append(category_text)
            questions.append(question)
            answers.append(answer)

    except Exception as e:
        print(f"❌ Tab ke-{i} gagal diproses: {e}")
        continue

# Buat DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({
    'category': categories,
    'question': questions,
    'answer': answers
})

# Tampilkan hasil
df


Total pertanyaan ditemukan: 28
Jumlah cards yang ditemukan: 28
Total pertanyaan ditemukan: 68
Jumlah cards yang ditemukan: 68
Total pertanyaan ditemukan: 4
Jumlah cards yang ditemukan: 4
Total pertanyaan ditemukan: 239
Jumlah cards yang ditemukan: 239
Total pertanyaan ditemukan: 3
Jumlah cards yang ditemukan: 3
Total pertanyaan ditemukan: 27
Jumlah cards yang ditemukan: 27


,category,question,answer
0,UMUM,Apa itu Allo Bank?,None
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,None
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?,None
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan aman?,None
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,None
...,...,...,...
364,REGISTRASI,Mengapa username dan Password saya diinfokan salah disaat saya telah menginput informasi yang benar?,None
365,REGISTRASI,Apa yang harus saya lakukan apabila saya tidak bisa menekan tombol 'Login'?,None
366,REGISTRASI,Apa yang harus saya lakukan apabila saat Login yang muncul hanya layar putih?,None
367,REGISTRASI,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.,None


In [24]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup Chrome WebDriver
driver = webdriver.Chrome()
url = "https://www.allobank.com/help"
driver.get(url)
wait = WebDriverWait(driver, 10)
time.sleep(5)  # Tunggu page awal load

categories = []
questions = []
answers = []

# Ambil semua tab kategori FAQ
tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")

for i in range(len(tab_buttons)):
    try:
        # Reload ulang tab untuk menghindari stale element
        tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")
        tab = tab_buttons[i]
        category_text = tab.text.strip()

        # Scroll dan klik tab
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        time.sleep(1)
        driver.execute_script("arguments[0].click();", tab)
        time.sleep(3)  # Tunggu konten tab termuat

        # Ambil semua elemen pertanyaan
        question_elements = driver.find_elements(By.CLASS_NAME, "cards__title")
        for q_el in question_elements:
            try:
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", q_el)
                driver.execute_script("arguments[0].click();", q_el)
                time.sleep(1.5)  # Tunggu animasi buka jawaban
            except Exception as e:
                print(f"❌ Gagal klik pertanyaan: {e}")
                continue

        # Ambil semua cards setelah pertanyaan diklik
        card_elements = driver.find_elements(By.CLASS_NAME, "cards")

        for card in card_elements:
            try:
                q_el = card.find_element(By.CLASS_NAME, "cards__title")
                a_el = card.find_element(By.CLASS_NAME, "cards__desc")
                q_text = q_el.text.strip()
                a_text = a_el.text.strip()

                questions.append(q_text)
                answers.append(a_text)
                categories.append(category_text)
            except Exception as e:
                print(f"❌ Gagal ekstrak Q&A: {e}")
                continue

    except Exception as e:
        print(f"❌ Tab ke-{i} gagal diproses: {e}")
        continue

# Buat DataFrame
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame({
    'category': categories,
    'question': questions,
    'answer': answers
})

# Tampilkan hasil
df


❌ Gagal ekstrak Q&A: Message: no such element: Unable to locate element: {"method":"css selector","selector":".cards__desc"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104c403cc cxxbridge1$str$ptr + 2829900
1   chromedriver                        0x0000000104c38694 cxxbridge1$str$ptr + 2797844
2   chromedriver                        0x0000000104775fbc cxxbridge1$string$len + 90140
3   chromedriver                        0x00000001047bd1bc cxxbridge1$string$len + 381468
4   chromedriver                        0x00000001047b29a0 cxxbridge1$string$len + 338432
5   chromedriver                        0x00000001047fe044 cxxbridge1$string$len + 647332
6   chromedriver                        0x00000001047b13f8 cxxbridge1$string$len + 332888
7   chromedriver                        0x0000000104

,category,question,answer
0,UMUM,Berapa persen CAR (Capital Adequacy Ratio) di Allo Bank?,Anda dapat melihat data terkait CAR (Capital Adequacy Ratio) dan data keuangan Allo Bank lainnya di laporan keuangan Allo Bank yang dapat Anda akses melalui halaman Hubungan Investor.
1,AKUN,Bagaimana cara saya menutup rekening Allo Prime?,
2,PRODUK,Mengapa saya tidak berhasil melakukan pembukaan Deposito?,
3,LAYANAN,Apakah saya dapat menggunakan kode referral yang telah digunakan oleh orang lain?,Anda dapat menggunakan kode referral yang telah digunakan oleh orang lain selama Anda belum pernah melakukan pembukaan rekening Allo Bank sebelumnya.
4,PROSES PENGADUAN NASABAH,Bagaimana Allo Bank menyelesaikan pengaduan dari Nasabah?,Allo Bank akan menyampaikan tanggapan atas pengaduan yang diberikan oleh Nasabah sesuai hasil pemeriksaan.\n\nAtas hasil pemeriksaan :\nJika Nasabah menerima hasil penyelesaian dan/atau tanggapan Allo Bank makan proses SELESAI.\nJika Nasabah tidak menerima hasil penyelesaian dan/atau tanggapan Allo Bank maka:\nNasabah dapat mengajukan keberatan dengan melampirkan dokumen baru yang belum disampaikan sebelumnya.\nDapat menyampaikan pengaduan melakui BI atau OJK\nMengupayakan penyelesaian sengketa baik di dalam maupun di luar pengadilan.\nNasabah dapat menyampaikan penyelesaian sengketa melalui Lembaga Alternatif Penyelesaian Sengketa Sektor Jasa Keuangan (LAPS SJK) yang telah mendapatkan ijin dari OJK. Konsumen dapat menghubungi LAPS SJK melalui :\nAlamat : Gedung Menara Karya lt. 25 Unit G-H Jl. HR. Rasuna Said Blok X-5 Kav 1-2 Jakarta 12950\nTelepon : 021-2527700\nEmail : info@lapssjk.id\nInformasi lebih lanjut mengenai LAPS SJK dapat diakses melalui https://lapssjk.id/
5,REGISTRASI,Mengapa nomor ponsel saya tidak terdaftar di Allo Bank padahal sebelumnya saya sudah melakukan registrasi?,"Pastikan nomor ponsel yang Anda masukkan sudah benar. Jika nomor ponsel sudah benar dan Anda masih mengalami kendala, Anda dapat menghubungi Allo Care kami untuk melakukan laporan melalui:\nTelepon: 080 4110 4110\nEmail: allocare@allobank.com\n\nSelanjutnya, setelah laporan sudah diterima akan dilakukan investigasi lebih lanjut."


In [25]:
df

,category,question,answer
0,UMUM,Berapa persen CAR (Capital Adequacy Ratio) di Allo Bank?,Anda dapat melihat data terkait CAR (Capital Adequacy Ratio) dan data keuangan Allo Bank lainnya di laporan keuangan Allo Bank yang dapat Anda akses melalui halaman Hubungan Investor.
1,AKUN,Bagaimana cara saya menutup rekening Allo Prime?,
2,PRODUK,Mengapa saya tidak berhasil melakukan pembukaan Deposito?,
3,LAYANAN,Apakah saya dapat menggunakan kode referral yang telah digunakan oleh orang lain?,Anda dapat menggunakan kode referral yang telah digunakan oleh orang lain selama Anda belum pernah melakukan pembukaan rekening Allo Bank sebelumnya.
4,PROSES PENGADUAN NASABAH,Bagaimana Allo Bank menyelesaikan pengaduan dari Nasabah?,Allo Bank akan menyampaikan tanggapan atas pengaduan yang diberikan oleh Nasabah sesuai hasil pemeriksaan.\n\nAtas hasil pemeriksaan :\nJika Nasabah menerima hasil penyelesaian dan/atau tanggapan Allo Bank makan proses SELESAI.\nJika Nasabah tidak menerima hasil penyelesaian dan/atau tanggapan Allo Bank maka:\nNasabah dapat mengajukan keberatan dengan melampirkan dokumen baru yang belum disampaikan sebelumnya.\nDapat menyampaikan pengaduan melakui BI atau OJK\nMengupayakan penyelesaian sengketa baik di dalam maupun di luar pengadilan.\nNasabah dapat menyampaikan penyelesaian sengketa melalui Lembaga Alternatif Penyelesaian Sengketa Sektor Jasa Keuangan (LAPS SJK) yang telah mendapatkan ijin dari OJK. Konsumen dapat menghubungi LAPS SJK melalui :\nAlamat : Gedung Menara Karya lt. 25 Unit G-H Jl. HR. Rasuna Said Blok X-5 Kav 1-2 Jakarta 12950\nTelepon : 021-2527700\nEmail : info@lapssjk.id\nInformasi lebih lanjut mengenai LAPS SJK dapat diakses melalui https://lapssjk.id/
5,REGISTRASI,Mengapa nomor ponsel saya tidak terdaftar di Allo Bank padahal sebelumnya saya sudah melakukan registrasi?,"Pastikan nomor ponsel yang Anda masukkan sudah benar. Jika nomor ponsel sudah benar dan Anda masih mengalami kendala, Anda dapat menghubungi Allo Care kami untuk melakukan laporan melalui:\nTelepon: 080 4110 4110\nEmail: allocare@allobank.com\n\nSelanjutnya, setelah laporan sudah diterima akan dilakukan investigasi lebih lanjut."


In [26]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Setup WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.allobank.com/help")
time.sleep(5)  # Tunggu halaman utama load

# Tempat simpan data
categories, questions, answers = [], [], []

# Ambil semua tab kategori FAQ
tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")

for i in range(len(tab_buttons)):
    try:
        # Reload ulang semua tab untuk menghindari stale element
        tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")
        tab = tab_buttons[i]
        category_text = tab.text.strip()

        print(f"\n📂 Kategori: {category_text}")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        driver.execute_script("arguments[0].click();", tab)
        time.sleep(3)  # Tunggu konten tab termuat

        # Ambil semua elemen pertanyaan
        question_elements = driver.find_elements(By.CLASS_NAME, "cards__title")

        for q_el in question_elements:
            try:
                q_text = q_el.text.strip()
                print(f"🔍 Proses pertanyaan: {q_text}")

                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", q_el)
                driver.execute_script("arguments[0].click();", q_el)

                # Tunggu jawaban muncul dalam card yang sama
                parent_card = q_el.find_element(By.XPATH, "./ancestor::div[contains(@class, 'cards')]")

                WebDriverWait(parent_card, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "cards__desc"))
                )

                a_el = parent_card.find_element(By.CLASS_NAME, "cards__desc")
                a_text = a_el.text.strip()

                # Simpan hasil
                categories.append(category_text)
                questions.append(q_text)
                answers.append(a_text)

                time.sleep(1.2)  # Tunggu animasi selesai

            except Exception as e:
                print(f"❌ Gagal proses pertanyaan: {e}")
                continue

    except Exception as e:
        print(f"❌ Gagal buka tab ke-{i}: {e}")
        continue

# Buat DataFrame
df = pd.DataFrame({
    'category': categories,
    'question': questions,
    'answer': answers
})

pd.set_option('display.max_colwidth', None)
print("\n✅ Ekstraksi selesai. Jumlah data:", len(df))
df



📂 Kategori: UMUM
🔍 Proses pertanyaan: Apa itu Allo Bank?
🔍 Proses pertanyaan: Kapan Allo Bank didirikan di Indonesia?
🔍 Proses pertanyaan: Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?
🔍 Proses pertanyaan: Apakah menyimpan dana di Allo Bank dipastikan aman?
🔍 Proses pertanyaan: Apakah Allo Bank memiliki kantor cabang?
🔍 Proses pertanyaan: Bagaimana jika saya ingin mengajukan pertanyaan atau keluhan kepada Allo Bank?
🔍 Proses pertanyaan: Di mana kantor pusat Allo Bank berada?
🔍 Proses pertanyaan: Apa itu aplikasi Allo Bank?
🔍 Proses pertanyaan: Fitur apa saja yang terdapat di dalam aplikasi Allo Bank?
🔍 Proses pertanyaan: Apa keuntungan menggunakan aplikasi Allo Bank?
🔍 Proses pertanyaan: Bagaimana cara mendapatkan aplikasi Allo Bank?
🔍 Proses pertanyaan: Apa yang harus saya lakukan setelah mengunduh aplikasi Allo Bank?
🔍 Proses pertanyaan: Apakah saya dapat log in ke 1 (satu) akun Allo Bank pada lebih dari 1 perangkat?
🔍 Proses pertanyaan: Apakah aplikasi Allo Ba

,category,question,answer
0,UMUM,Apa itu Allo Bank?,"Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan ""All in One"" dan mempunyai arti sebagai ""one for all, and all for one"" alias semua untuk satu dan satu untuk semua."
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?,
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan aman?,
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,
...,...,...,...
364,REGISTRASI,Mengapa username dan Password saya diinfokan salah disaat saya telah menginput informasi yang benar?,
365,REGISTRASI,Apa yang harus saya lakukan apabila saya tidak bisa menekan tombol 'Login'?,
366,REGISTRASI,Apa yang harus saya lakukan apabila saat Login yang muncul hanya layar putih?,
367,REGISTRASI,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.,


In [27]:
df['category'].unique()

array(['UMUM', 'AKUN', 'PRODUK', 'LAYANAN', 'PROSES PENGADUAN NASABAH',
       'REGISTRASI'], dtype=object)

In [29]:
# Misalnya data kamu sudah dibaca ke dalam DataFrame bernama df
df_registrasi = df[df['category'] == 'REGISTRASI']
df_registrasi


,category,question,answer
342,REGISTRASI,Siapa saja yang dapat membuka rekening di aplikasi Allo Bank?,
343,REGISTRASI,"Saya tinggal di luar negeri, apakah saya bisa registrasi dan menggunakan aplikasi Allo?",
344,REGISTRASI,OS versi apa saja yang didukung oleh aplikasi Allo ?,
345,REGISTRASI,Bagaimana jika Smartphone yang saya miliki masih belum mendukung untuk menggunakan aplikasi Allo?,
346,REGISTRASI,Bagaimana cara saya mengunduh aplikasi Allo?,
347,REGISTRASI,Apakah ada biaya tambahan yang dikenakan saat proses registrasi akun Allo Bank?,
348,REGISTRASI,Apakah saya perlu memakai nomor handpone dari operator tertentu untuk melakukan registrasi Allo Bank?,
349,REGISTRASI,Mengapa saya perlu memasukkan email saat melakukan pembukaan rekening Allo Bank?,
350,REGISTRASI,Apa saja kriteria format PIN yang ditentukan pada aplikasi Allo?,
351,REGISTRASI,Apakah saya bisa menggunakan nomor ponsel atau alamat email yang sudah terdaftar di aplikasi Allo untuk melakukan registrasi?,


In [32]:
df_umum = df[df['category'] == 'UMUM']
df_umum

,category,question,answer
0,UMUM,Apa itu Allo Bank?,"Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan ""All in One"" dan mempunyai arti sebagai ""one for all, and all for one"" alias semua untuk satu dan satu untuk semua."
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?,
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan aman?,
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,
5,UMUM,Bagaimana jika saya ingin mengajukan pertanyaan atau keluhan kepada Allo Bank?,
6,UMUM,Di mana kantor pusat Allo Bank berada?,
7,UMUM,Apa itu aplikasi Allo Bank?,
8,UMUM,Fitur apa saja yang terdapat di dalam aplikasi Allo Bank?,
9,UMUM,Apa keuntungan menggunakan aplikasi Allo Bank?,


In [ ]:
df_umum = df[df['category'] == 'UMUM']
df_umum

In [31]:
for kategori in ['REGISTRASI']:
    print(f"\nKategori: {kategori}")
    df[df['category'] == kategori]


Kategori: REGISTRASI


In [33]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

# Setup WebDriver
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
driver.get("https://www.allobank.com/help")
time.sleep(5)  # Tunggu halaman utama load

# Tempat simpan data
categories, questions, answers = [], [], []

# Setup ActionChains untuk klik dengan stabil
actions = ActionChains(driver)

# Ambil semua tab kategori FAQ
tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")

for i in range(len(tab_buttons)):
    try:
        # Reload ulang semua tab untuk menghindari stale element
        tab_buttons = driver.find_elements(By.CLASS_NAME, "help__tabs__container__tab")
        tab = tab_buttons[i]
        category_text = tab.text.strip()

        print(f"\n📂 Kategori: {category_text}")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
        driver.execute_script("arguments[0].click();", tab)
        time.sleep(3)  # Tunggu konten tab termuat

        # Ambil semua elemen pertanyaan
        question_elements = driver.find_elements(By.CLASS_NAME, "cards__title")

        for q_el in question_elements:
            try:
                q_text = q_el.text.strip()
                print(f"🔍 Proses pertanyaan: {q_text}")

                # Scroll ke elemen dan klik pertanyaan
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", q_el)
                actions.move_to_element(q_el).click().perform()

                # Tunggu beberapa detik agar jawaban terbuka
                time.sleep(1.5)  # Tunggu animasi

                # Cek apakah ada jawaban terbuka dan tutup jawaban sebelumnya
                open_answer = driver.find_elements(By.CLASS_NAME, "cards__desc")
                if open_answer:
                    print("📝 Jawaban terbuka, menunggu konten baru...")
                
                # Tunggu jawaban muncul pada card yang dipilih
                parent_card = q_el.find_element(By.XPATH, "./ancestor::div[contains(@class, 'cards')]")
                
                # Tunggu hingga konten jawaban ada
                WebDriverWait(parent_card, 5).until(
                    lambda d: parent_card.find_element(By.CLASS_NAME, "cards__desc").text.strip() != ""
                )

                a_el = parent_card.find_element(By.CLASS_NAME, "cards__desc")
                a_text = a_el.text.strip()

                # Simpan hasil
                categories.append(category_text)
                questions.append(q_text)
                answers.append(a_text)

                time.sleep(1.2)  # Tunggu animasi selesai sebelum melanjutkan

            except Exception as e:
                print(f"❌ Gagal proses pertanyaan: {e}")
                continue

    except Exception as e:
        print(f"❌ Gagal buka tab ke-{i}: {e}")
        continue

# Buat DataFrame
df = pd.DataFrame({
    'category': categories,
    'question': questions,
    'answer': answers
})

pd.set_option('display.max_colwidth', None)
print("\n✅ Ekstraksi selesai. Jumlah data:", len(df))
df



📂 Kategori: UMUM
🔍 Proses pertanyaan: Apa itu Allo Bank?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Kapan Allo Bank didirikan di Indonesia?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Apakah menyimpan dana di Allo Bank dipastikan aman?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Apakah Allo Bank memiliki kantor cabang?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Bagaimana jika saya ingin mengajukan pertanyaan atau keluhan kepada Allo Bank?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Di mana kantor pusat Allo Bank berada?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Apa itu aplikasi Allo Bank?
📝 Jawaban terbuka, menunggu konten baru...
🔍 Proses pertanyaan: Fitur apa saja yang terdapat di dalam aplikasi Allo Bank?
📝 Jawaban terbuka, menung

,category,question,answer
0,UMUM,Apa itu Allo Bank?,"Sebuah layanan perbankan berbasis digital yang dapat memudahkan setiap nasabahnya dalam melakukan kegiatan finansial ataupun non finansial pada satu aplikasi. Bagi yang belum tahu, nama dari Allo Bank sebenarnya memiliki singkatan ""All in One"" dan mempunyai arti sebagai ""one for all, and all for one"" alias semua untuk satu dan satu untuk semua."
1,UMUM,Kapan Allo Bank didirikan di Indonesia?,Allo Bank didirikan di Indonesia pada tahun 2021.
2,UMUM,Apakah Allo Bank terdaftar dan diawasi oleh Otoritas Jasa Keuangan?,Benar! Allo Bank telah terdaftar dan diawasi oleh Otoritas Jasa Keuangan (OJK).
3,UMUM,Apakah menyimpan dana di Allo Bank dipastikan aman?,Tentu saja! Allo Bank terdaftar sebagai peserta Lembaga Penjamin Simpanan (LPS) serta terdaftar dan diawasi oleh Otoritas Jasa Keuangan (OJK).
4,UMUM,Apakah Allo Bank memiliki kantor cabang?,"Saat ini, Allo Bank memiliki dua kantor cabang yang berlokasi di:\n1. Menara Bank Mega, Jl. Kapten Tendean No.12 Mampang Prapatan, South Jakarta 12790 Indonesia\n2. Indofood Tower, South Jakarta 12790 Indonesia"
...,...,...,...
364,REGISTRASI,Mengapa username dan Password saya diinfokan salah disaat saya telah menginput informasi yang benar?,"Pastikan password yang dimasukkan sudah benar. Jika masih belum berhasil, Anda dapat menekan tombol ""Lupa Password"" dan mengatur ulang password Anda."
365,REGISTRASI,Apa yang harus saya lakukan apabila saya tidak bisa menekan tombol 'Login'?,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.
366,REGISTRASI,Apa yang harus saya lakukan apabila saat Login yang muncul hanya layar putih?,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.
367,REGISTRASI,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.,Anda dapat menutup aplikasi Allo Bank dan membukanya kembali. Pastikan juga Anda terhubung dengan jaringan internet yang stabil dan coba lagi.
